In [1]:
from code.util import *

In [2]:
data = load("data/urls.pkl")

In [3]:
data1 = load("data/url_1.pkl")

In [11]:
data = data[data.target==0]

In [12]:
newdata = pd.concat([data, data1])

In [13]:
newdata.head()

,target,url
0,0,http://www.liquidgeneration.com/
1,0,http://www.onlineanime.org/
2,0,http://www.ceres.dti.ne.jp/~nekoi/senno/senfir...
3,0,http://www.galeon.com/kmh/
4,0,http://www.fanworkrecs.com/


In [14]:
newdata.tail()

,target,url
22421,1,http://www.gkjx168.com/images/
22422,1,http://www.gkjx168.com/images/?ref=http://uqpz...
22423,1,http://mymvm.com/public/public/public/_cache/i...
22424,1,http://www.mymvm.com/public/public/public/_cac...
22425,1,http://paypal.com.cgi-bin.login.submited.elkkl...


In [18]:
len(newdata[newdata.target==0])

1048576

In [19]:
len(newdata[newdata.target==1])

22426

In [21]:
save(newdata, "data/urls.pkl")

In [22]:
data = load("data/urls.pkl")

In [23]:
len(data)

1071002

In [24]:
len(data[newdata.target==0])

1048576

In [25]:
len(data[newdata.target==1])

22426